In [94]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import os

from matplotlib.collections import LineCollection

from download_data import download_googledrive_folder

from config import *

In [3]:



def colored_line(x, y, c, ax, **lc_kwargs):
    """
    Plot a line with a color specified along the line by a third value.

    It does this by creating a collection of line segments. Each line segment is
    made up of two straight lines each connecting the current (x, y) point to the
    midpoints of the lines connecting the current point with its two neighbors.
    This creates a smooth line with no gaps between the line segments.

    Parameters
    ----------
    x, y : array-like
        The horizontal and vertical coordinates of the data points.
    c : array-like
        The color values, which should be the same size as x and y.
    ax : Axes
        Axis object on which to plot the colored line.
    **lc_kwargs
        Any additional arguments to pass to matplotlib.collections.LineCollection
        constructor. This should not include the array keyword argument because
        that is set to the color argument. If provided, it will be overridden.

    Returns
    -------
    matplotlib.collections.LineCollection
        The generated line collection representing the colored line.
    """
    if "array" in lc_kwargs:
        warnings.warn('The provided "array" keyword argument will be overridden')

    # Default the capstyle to butt so that the line segments smoothly line up
    default_kwargs = {"capstyle": "butt"}
    default_kwargs.update(lc_kwargs)

    # Compute the midpoints of the line segments. Include the first and last points
    # twice so we don't need any special syntax later to handle them.
    x = np.asarray(x)
    y = np.asarray(y)
    x_midpts = np.hstack((x[0], 0.5 * (x[1:] + x[:-1]), x[-1]))
    y_midpts = np.hstack((y[0], 0.5 * (y[1:] + y[:-1]), y[-1]))

    # Determine the start, middle, and end coordinate pair of each line segment.
    # Use the reshape to add an extra dimension so each pair of points is in its
    # own list. Then concatenate them to create:
    # [
    #   [(x1_start, y1_start), (x1_mid, y1_mid), (x1_end, y1_end)],
    #   [(x2_start, y2_start), (x2_mid, y2_mid), (x2_end, y2_end)],
    #   ...
    # ]
    coord_start = np.column_stack((x_midpts[:-1], y_midpts[:-1]))[:, np.newaxis, :]
    coord_mid = np.column_stack((x, y))[:, np.newaxis, :]
    coord_end = np.column_stack((x_midpts[1:], y_midpts[1:]))[:, np.newaxis, :]
    segments = np.concatenate((coord_start, coord_mid, coord_end), axis=1)

    lc = LineCollection(segments, **default_kwargs)
    lc.set_array(c)  # set the colors of each segment

    return ax.add_collection(lc)


def plot_curve(X):
    """
    Plot a curve given by the points in X.

    Parameters
    ----------
    X : torch.Tensor
        The points on the curve. Should be a 2D tensor with shape (n_points, 3).
        each point represents a (t, x, y) value.
    """

    # -------------- Create and show plot --------------
    # Some arbitrary function that gives x, y, and color values
    t = X[:, 0].numpy()
    x = X[:, 1].numpy()
    y = X[:, 2].numpy()
    color = np.linspace(0, int(np.max(t)) + 1,t.size )  # color by t value

    # Create a figure and plot the line on it
    fig1, ax1 = plt.subplots()
    lines = colored_line(x, y, color, ax1,  cmap="plasma")
    fig1.colorbar(lines)  # add a color legend

    # Set the axis limits and tick positions
    ax1.set_xlim(np.min(x), np.max(x))
    ax1.set_ylim(np.min(y), np.max(y))
    # ax1.set_xticks((-1, 0, 1))
    # ax1.set_yticks((-1, 0, 1))
    ax1.set_title("worm movement")

    plt.show()

In [ ]:


def load_data(data_path):
    # list all csv files in the directory
    csv_files = os.listdir(data_path)
    csv_files = [f for f in csv_files if f.endswith(".csv")]

    worms=[]
    min_length = 9999999999
    for worm in csv_files:
        worm_data = np.genfromtxt(data_path+worm, delimiter=",", skip_header=1, dtype=np.float32)
        min_length = min(worm_data.shape[0], min_length)
        worms.append(worm_data)
    return worms, min_length


control_data,min_length1 = load_data("./data/Lifespan/control/")
drug_data,min_length2 = load_data("./data/Lifespan/companyDrug/")
min_length = min(min_length1,min_length2)
num_control = len(control_data)
num_drug = len(drug_data)

worms = control_data + drug_data
lifespan = np.zeros(len(worms))
for i in range(len(worms)):
    worm_xy = worms[i][:,[2,3]]
    delta_xy = np.diff(worm_xy, axis=0)
    speed = np.linalg.norm(delta_xy, axis=1)
    inactive_frames = speed<DEATH_THRESHOLD
    cum_inactive_frames = np.cumsum(inactive_frames, axis=0)
    consecu_inactive = cum_inactive_frames-np.roll(cum_inactive_frames, DEATH_LENGTH, axis=0)
    dead = consecu_inactive==(DEATH_LENGTH-1)
    # get the first dead frame
    res = np.where(dead)[0]
    if res.shape[0] == 0:
        lifespan[i] = worm_xy.shape[0]//900/4
    else:
        lifespan[i] = res[0]//900/4

    
    worms[i] = worms[i][:min_length]
data = np.stack(worms, axis=0)
t_xy = torch.tensor(data[:, :, [0,2,3]])
xy = t_xy[:, :, 1:]

x = xy
# y = torch.tensor([0]*num_control + [1]*num_drug, dtype=torch.float32)
y = torch.tensor(lifespan, dtype=torch.float32)


In [58]:
y

tensor([20.2500, 19.2500, 13.7500, 16.0000, 17.7500, 17.7500, 14.2500, 15.0000,
        14.7500, 13.0000, 15.0000, 15.0000, 14.7500,  0.5000, 12.5000, 17.5000,
        12.2500, 18.7500, 16.7500, 13.7500, 20.7500, 12.7500, 16.5000, 17.7500])

In [59]:
x=xy[:,:900*8].reshape(x.shape[0],-1,900,2)

In [60]:
x.shape

torch.Size([24, 8, 900, 2])

In [61]:
last_frame = torch.roll(x, 1, 2) 
delta_xy = (x - last_frame)[:,:,1:,:]
speed = torch.norm(delta_xy, dim=3)
avg_speed_single_period = torch.nanmean(speed, dim=2)
avg_speed = torch.nanmean(avg_speed_single_period, dim=1)
# print(avg_speed_single_period)

def avg_first_k_periods(x, k):
    
    avg_single_period = torch.mean(x, dim=2)
    avg = torch.nanmean(avg_single_period[:,:k], dim=1)
    return avg

def var_single_period(x):
    return torch.var(x, dim=2)
     
def var_first_k_periods(x, k):
    avg_single_period = torch.mean(x, dim=2)
    avg = torch.var(avg_single_period[:,:k], dim=1)
    return avg

def avg_across_periods(x,start,end):
    avg_single_period = torch.mean(x, dim=2)
    var = torch.nanmean(avg_single_period[:,start:end], dim=1)
    return var

# avg_speed_first_4_periods = avg_first_k_periods(speed, 4)
avg_speed_first_4_periods = avg_speed
avg_speed_first_4_periods



tensor([1.7086, 1.9527, 2.1065, 1.6648, 1.4510, 1.9013, 2.3264, 1.2639, 2.1138,
        8.7371, 1.4162, 1.4512, 4.3873, 0.9761, 1.5847, 7.2195, 1.5256, 2.7563,
        1.7761, 1.3338, 1.8913, 1.5563, 1.1253, 1.6555])

In [62]:
acceleration = speed - torch.roll(speed, 1, 2)

# avg_acc_first_2_periods = avg_first_k_periods(acceleration, 2)

avg_acc = torch.nanmean(torch.nanmean(acceleration, dim=2), dim=1)

In [63]:
threshold = 0
active_frames = speed > threshold
active_time = torch.sum(active_frames, dim=2)
# divide active_time by number of non-nan frames
active_time = active_time / (active_frames.shape[2] - torch.sum(torch.isnan(speed), dim=2))
avg_active_time = torch.nanmean(active_time, dim=1,dtype=torch.float32)

avg_active_time


tensor([0.8879, 0.7807, 0.9982, 0.9546, 0.9892, 0.9963, 0.9787, 0.9965, 0.9889,
        0.9218, 0.9537, 0.9691, 0.9105, 0.7421, 0.9932, 0.9355, 0.9634, 0.9420,
        0.9947, 0.8640, 0.8967, 0.9932, 0.9091, 0.9939])

In [64]:
sharp_turn = torch.sum(torch.sum(delta_xy*torch.roll(delta_xy, 1, 2),dim=3)[:,:,2:]<0,dim=2)

# divide sharp_turn by number of non-nan frames
sharp_turn = sharp_turn / (delta_xy.shape[2] - torch.sum(torch.any(torch.isnan(delta_xy),dim=3), dim=2))
avg_sharp_turn = torch.nanmean(sharp_turn, dim=1,dtype=torch.float32)

In [65]:
x_shifted = x-x[:,:,0,:].unsqueeze(2)
rotation_angle = -torch.arctan(x_shifted[:,:,-1,1]/x_shifted[:,:,-1,0]).unsqueeze(2)
# angles = torch.arctan(x_shifted[:,:,:,1]/x_shifted[:,:,:,0])
# rotation_angles = end_angle - angles
cos_rot = torch.cos(rotation_angle).unsqueeze(3)
sin_rot = torch.sin(rotation_angle).unsqueeze(3)
rotated_x = torch.sum(x_shifted.unsqueeze(4)*torch.cat((cos_rot, -sin_rot,sin_rot,cos_rot), dim=3).reshape(x_shifted.shape[0],x_shifted.shape[1],1,2,2),axis=4)

final_x=torch.where(torch.isnan(rotated_x), x_shifted, rotated_x)

In [66]:
#fill nan on final x by using the next and last frame along dimension 2
final_x_filled = torch.where(torch.isnan(final_x), torch.roll(final_x, 1, 2), final_x)
final_x_filled = torch.where(torch.isnan(final_x_filled), torch.roll(final_x_filled, -1, 2), final_x_filled)


In [67]:
from shapely import LineString, frechet_distance

frechet_dist = torch.zeros(x.shape[0], x.shape[1]-1)
for i in range(x.shape[0]):
    for period in range(x.shape[1]-1):
        frechet_dist[i][period] = frechet_distance(LineString(final_x_filled[i][period]), LineString(final_x_filled[i][period+1]))

frechet_dist


c:\Users\danie\OneDrive - HKUST Connect\CS433\MLProject2\.venv\Lib\site-packages\shapely\measurement.py:261: RuntimeWarning: invalid value encountered in frechet_distance
  return lib.frechet_distance(a, b, **kwargs)


tensor([[168.5555,  43.6669,      nan,      nan, 431.8122, 436.9256,  44.0135],
        [ 56.8308,   0.9308,   3.3427,  35.1488, 158.0401, 493.9297, 405.7979],
        [     nan,      nan, 212.3379,  78.7844, 487.1808, 542.5905, 328.4906],
        [     nan,   9.6561,  39.0992, 154.1058, 167.5956,      nan,      nan],
        [ 52.8693, 257.3020, 292.4625, 141.8313, 100.8104, 311.4603, 299.4502],
        [137.8762, 157.0831, 287.4040, 140.2231,  83.1240, 230.4048, 712.6235],
        [     nan, 113.3740, 131.1903,  70.3422,  12.1923, 169.1046, 343.7606],
        [     nan,  43.8303,  60.3129, 333.9238, 328.2848, 303.3276, 280.7503],
        [ 14.0989, 231.5280, 205.9086, 230.7948, 326.9601, 161.9845, 148.4532],
        [352.1542, 416.6798, 265.9317,      nan,      nan, 292.3742, 399.4318],
        [258.3849, 160.4782, 350.2946, 287.5157,  46.6903, 111.8296, 354.4059],
        [454.3577, 478.5380, 373.1893, 172.0591, 286.5813, 172.0199, 171.1631],
        [332.1151, 347.7637, 221.1542, 3

In [68]:
features = torch.stack((avg_speed_first_4_periods, avg_acc, avg_active_time, avg_sharp_turn, torch.nanmean(frechet_dist,dim=1)), dim=1)
features.shape

torch.Size([24, 5])

In [69]:
def normalize(features):
    return (features - torch.mean(features, dim=0))/torch.std(features, dim=0)
tx = normalize(features)


In [85]:
# normalize y

ty = (y - torch.mean(y))/torch.std(y)

In [70]:
# linear classifier

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

X_train, X_test, y_train, y_test = train_test_split(tx, y, test_size=0.33, random_state=42)

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

# confusion_matrix(y_test, y_pred)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [ ]:
# random forest
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
# accuracy_scores = []
# f1_scores = []
# for i in tqdm(range(1000)):
#     X_train, X_test, y_train, y_test = train_test_split(tx, y, test_size=0.33, random_state=i)
#     clf = RandomForestClassifier(max_depth=2)
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
#     accuracy_scores.append(accuracy_score(y_test, y_pred))
#     f1_scores.append(f1_score(y_test, y_pred))

# print(np.mean(accuracy_scores))
# print(np.mean(f1_scores))
clf = RandomForestClassifier(max_depth=2)
clf.fit(X_train[:,[0]], y_train)
y_pred = clf.predict(X_test[:,[0]])
print(accuracy_score(y_test, y_pred))

print(f1_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)

clf.feature_importances_

# train accuracy
# y_pred = clf.predict(X_train)
# print(accuracy_score(y_train, y_pred))

100%|██████████| 1000/1000 [02:23<00:00,  6.98it/s]

0.37875
0.35925519480519486


array([0.192431  , 0.23376084, 0.19435263, 0.1117646 , 0.26769092])

In [ ]:
# SVM classifier
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

confusion_matrix(y_test, y_pred)

# train accuracy
y_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_pred))

0.375
0.2857142857142857
0.8125


In [ ]:
# knn classifier
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(f1_score(y_test, y_pred))


# train accuracy
y_pred = clf.predict(X_train)
print(accuracy_score(y_train, y_pred))

0.375
0.2857142857142857
0.625


In [87]:
# regression models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split

# metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(tx,ty, test_size=0.33)

def evaluate_regression_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"MSE: {mse}")
    print(f"R2: {r2}")


In [88]:
# linear regression

reg = LinearRegression().fit(X_train, y_train)

evaluate_regression_model(reg, X_train, y_train, X_test, y_test)

MSE: 1.5023913383483887
R2: -3.85640811920166


In [89]:
# SVR
reg = SVR().fit(X_train, y_train)

evaluate_regression_model(reg, X_train, y_train, X_test, y_test)

MSE: 0.9063017547647877
R2: -1.9295769815962882


In [90]:
# random forest
reg = RandomForestRegressor(max_depth=3).fit(X_train, y_train)


evaluate_regression_model(reg, X_train, y_train, X_test, y_test)
reg.feature_importances_

MSE: 0.6998694029877301
R2: -1.2622942991526154


array([0.19798165, 0.08623673, 0.31376139, 0.35061783, 0.0514024 ])

In [91]:
# gaussian process
kernel = 1.0 * RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
reg = GaussianProcessRegressor(kernel=kernel, random_state=0)

evaluate_regression_model(reg, X_train, y_train, X_test, y_test)



MSE: 1.3548211804193184
R2: -3.3793945266779657


c:\Users\danie\OneDrive - HKUST Connect\CS433\MLProject2\.venv\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


In [92]:
# neural network
reg = MLPRegressor(random_state=1, max_iter=500)

evaluate_regression_model(reg, X_train, y_train, X_test, y_test)

MSE: 3.1432291391381177
R2: -9.160337531611184
